In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, recall_score, accuracy_score
import seaborn as sns
# warnings.filterwarnings('ignore')


In [2]:
df = pd.read_csv("data/sqli.csv", encoding='utf-16')
print("Data Shape:", df.shape)
df.Label.unique()
df[df['Label']==0]

Data Shape: (4200, 2)


,Sentence,Label
1128,Add plain text Ne te quaesiveris extra,0
1129,"Man star; soul Render honest perfect man, Com...",0
1130,"Our acts angels are, good ill, Our fatal shad...",0
1131,Epilogue Beaumont Fletcher’s Honest Man’s For...,0
1132,I read day verses written eminent painter ori...,0
...,...,...
4195,"org/?option = com_k2 <a href = ""http://corfopym",0
4196,com/?option = com_k2 <act> <![CDATA[procMemb...,0
4197,picsearch,0
4198,com/is?-WZx-uhyLezKNiYLvAbKL3W4oh5F749nr2KUmFF...,0


In [3]:
df.head()

,Sentence,Label
0,a,1
1,a',1
2,a' --,1
3,a' or 1 = 1; --,1
4,@,1


In [4]:
X = df['Sentence']
x_inp1 = X[3]
x_inp1

"a' or 1 = 1; --"

In [6]:
# X = df['Sentence']
# y = df['Label']
# vectorizer = CountVectorizer(min_df = 2, max_df = 0.8, stop_words = stopwords.words('english'))
# X = vectorizer.fit_transform(X.values.astype('U')).toarray()
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

# single_string = [x_inp1]
# single_string_vector = vectorizer.transform(single_string).toarray()
# single_string_tensor = np.array(single_string_vector, dtype=np.float32)
# model.predict(single_string_tensor)

In [7]:
X = df['Sentence']
y = df['Label']
print(X.shape, y.shape)
len(X)
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
nltk.download('stopwords')
vectorizer = CountVectorizer(min_df = 2, max_df = 0.8, stop_words = stopwords.words('english'))
X = vectorizer.fit_transform(X.values.astype('U')).toarray()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(4200,) (4200,)
(3360, 4717)
(3360,)
(840, 4717)
(840,)


[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


In [8]:
X[1]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

### CNN Model

In [9]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras import models, layers
# import warnings

from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import precision_recall_curve,precision_recall_fscore_support


model = models.Sequential()
model.add(layers.Conv1D(32, 1, activation = 'relu', input_shape = (1,4717)))
model.add(layers.Conv1D(32, 1, activation = 'relu'))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation = 'sigmoid'))
model.summary()
model.compile(optimizer = 'adam', loss = tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])

X_train1 = X_train.reshape(-1, 1, 4717)
X_test1 = X_test.reshape(-1, 1, 4717)

history = model.fit(X_train1, y_train, epochs = 10, validation_data = (X_test1, y_test))
y_pred = model.predict(X_test1).flatten()

y_pred = np.round(y_pred)

model.save("saveModel/")


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1, 32)             150976    
                                                                 
 conv1d_1 (Conv1D)           (None, 1, 32)             1056      
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 152,065
Trainable params: 152,065
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
105/105 [==============================] - 1s 7ms/step - loss: 0.4200 - accuracy: 0.8274 - val_loss: 0.1920 - val_accuracy: 0.9048
Epoch 2/10
105/105 [====================

INFO:tensorflow:Assets written to: saveModel/assets


INFO:tensorflow:Assets written to: saveModel/assets


In [50]:
# X = df['Sentence']
# y = df['Label']
# vectorizer = CountVectorizer(min_df = 2, max_df = 0.8, stop_words = stopwords.words('english'))
# X = vectorizer.fit_transform(X.values.astype('U')).toarray()
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

# single_string = [x_inp1]
# single_string_vector = vectorizer.transform(single_string).toarray()
# single_string_tensor = np.array(single_string_vector, dtype=np.float32)
# model.predict(single_string_tensor.reshape(-1,1,4717))

1/1 [==============================] - 0s 253ms/step


array([[0.84493566]], dtype=float32)

In [10]:
print(f"Accuracy of CNN on test set : {accuracy_score(y_pred, y_test)}")
print(f"F1 Score of CNN on test set : {f1_score(y_pred, y_test)}")
print(f"recall of CNN on test set : {recall_score(y_pred, y_test)}")
print(f"precision of CNN on test set : {precision_score(y_pred, y_test)}")


Accuracy of CNN on test set : 0.9773809523809524
F1 Score of CNN on test set : 0.9616161616161617
recall of CNN on test set : 0.9296875
precision of CNN on test set : 0.99581589958159


In [16]:
import random
x_inp = X[random.randint(0,len(X)-1)]
# np.round(model.predict(x_inp.reshape(-1,1,4717)).flatten())
model.predict(x_inp.reshape(-1,1,4717)).flatten()

1/1 [==============================] - 0s 25ms/step


array([2.2971408e-05], dtype=float32)

In [19]:
# from tensorflow.keras.models import model_from_json
# model_json = model.to_json()
# with open("cnn_model.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights("cnn_model.h5")
# print("Saved model to disk")


Saved model to disk


In [17]:
from tensorflow.keras.models import model_from_json
json_file = open('cnn_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
cnn_model = model_from_json(loaded_model_json)
# load weights into new model
cnn_model.load_weights("cnn_model.h5")
print("Loaded model from disk")

# Use the loaded model to make predictions
cnn_model.predict(x_inp.reshape(-1,1,4717)).flatten()

Loaded model from disk
1/1 [==============================] - 0s 84ms/step


array([1.519039e-05], dtype=float32)

# RL MODEL

### Q Learning

In [21]:
import random
random_index = random.randint(0,len(X)-1)
def generate_random_query():
    return X[random.randint(0,len(X)-1)]
def is_injection(xi):
    # x_inp = X[random.randint(0,len(X)-1)]
    return model.predict(xi.reshape(-1,1,4717)).flatten()
def apply_action_to_query():
    return X[random.randint(0,len(X)-1)]

In [22]:
import numpy as np
import gym

class QLearningBinaryClassifier:
    def __init__(self, num_features, learning_rate, discount_factor):
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.weights = np.zeros(num_features)

    def predict(self, state):
        q_value = np.dot(self.weights, state)
        action = 1 if q_value >= 0 else 0
        return action

    def update(self, state, action, reward, next_state):
        q_value = np.dot(self.weights, state)
        next_q_value = np.dot(self.weights, next_state)
        target = reward + self.discount_factor * next_q_value
        error = target - q_value
        self.weights += self.learning_rate * error * state

def generate_random_query():
    return X[random.randint(0,len(X)-1)]
def is_injection(xi):
    # x_inp = X[random.randint(0,len(X)-1)]
    return model.predict(xi.reshape(-1,1,4717)).flatten()

# Define the RL agent (QLearining model )
class SQLInjectionAgent:
    def __init__(self):
        self.model = QLearningBinaryClassifier(num_features=5, learning_rate = 0.1, discount_factor = 0.9)  # Define the RL model (e.g., Q-learning, SARSA)

    def act(self, state):
        action = self.model.predict(state)  # Use the model to predict the best action
        return action

class SQLInjectionEnv(gym.Env):
    
    def __init__(self):
        self.state = None
        self.done = False
        # self.index = 0

    def reset(self):
        # Generate a new random SQL query to use as the initial state
        # self.state = generate_random_query()
        self.done = False
        # return self.state

    def step(self, state, action):
        # Execute the query with the given action (e.g., input, modification)
        # query = apply_action_to_query(self.state, action)
        # self.index = self.index+1
        query = state
        # query = X_train[self.index]
        result = is_injection(query)

        # Determine the reward based on the result of the query
        if is_injection(query):
            reward = -1  # Penalty for a successful injection
            self.done = True
        elif result == 'error':
            reward = -0.5  # Penalty for a failed query
        else:
            reward = 0.5  # Reward for a successful query

        return query, reward, self.done, {}

# Train the RL agent
env = SQLInjectionEnv()
agent = SQLInjectionAgent()

num_features = 4717
learning_rate = 0.1
discount_factor = 0.9
q_model = QLearningBinaryClassifier(num_features, learning_rate, discount_factor)

out = []

# num_episodes = 10**2
num_episodes = len(X_train)-1
# for episode in range(num_episodes):
for state in X_train:
    # state = env.reset() # Initialize the state
    done = False
    while not done:
        action = q_model.predict(state)
        out.append(action)
        next_state, reward, done, _ = env.step(state,action) # Observe the next state, reward, and done flag
        q_model.update(state, action, reward, next_state)
        state = next_state


1/1 [==============================] - 0s 23ms/step


In [29]:
import pickle
with open("ql_model.pkl", "wb") as f:
    pickle.dump(model, f)


INFO:tensorflow:Assets written to: ram://a6af2e1d-2a50-4b1d-b1b6-ee1985ec43db/assets


INFO:tensorflow:Assets written to: ram://a6af2e1d-2a50-4b1d-b1b6-ee1985ec43db/assets


In [23]:
from tensorflow.keras.models import model_from_json
model_json = model.to_json()
with open("q_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("q_model.h5")
print("Saved model to disk")



Saved model to disk


In [25]:
from tensorflow.keras.models import model_from_json

json_file = open('q_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
q_model = model_from_json(loaded_model_json)
# load weights into new model
q_model.load_weights("q_model.h5")
print("Loaded model from disk")


Loaded model from disk


In [26]:
x_inp = X[1128]
q_model.predict( x_inp )

ValueError: in user code:

    File "c:\Python310\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "c:\Python310\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Python310\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Python310\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "c:\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Python310\lib\site-packages\keras\engine\input_spec.py", line 250, in assert_input_compatibility
        raise ValueError(

    ValueError: Exception encountered when calling layer "sequential_1" "                 f

In [24]:
y_pred = []

for state in X_test:
    # state = env.reset() # Initialize the state
    done = False
    while not done:
        action = q_model.predict(state)
        y_pred.append(action)
        next_state, reward, done, _ = env.step(state,action) # Observe the next state, reward, and done flag
        q_model.update(state, action, reward, next_state)
        state = next_state

1/1 [==============================] - 0s 20ms/step


In [25]:
len(y_pred)

840

In [26]:
# x_inp = X[random.randint(0,len(X)-1)]
x_inp = X[1128]
# x_inp = X[1]
X_test1

array([[[0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0]]], dtype=int64)

In [30]:
type(x_inp)

numpy.ndarray

In [27]:
q_model.predict( x_inp )

0

In [28]:
model.predict(x_inp.reshape(-1,1,4717)).flatten()

1/1 [==============================] - 0s 23ms/step


array([0.9462793], dtype=float32)

In [29]:
print(len(y_test))
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


840
Accuracy: 0.763095238095238


In [30]:
y_pred1 = []
for state in X_test:
    action = q_model.predict(state)
    y_pred1.append(action)

accuracy = accuracy_score(y_test, y_pred1)
print("Accuracy:", accuracy)


Accuracy: 0.7642857142857142


### Deep Q Learning